# Image Classification

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import zipfile

## Cat & Dog Image Classification Using CNNs & TensorFlow

### Import Data

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /content/cats_and_dogs_filtered.zip

### Extract Data and Build Directory Structure

In [ ]:
local_zip = "cats_and_dogs_filtered.zip"
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('cats_and_dogs_filtered')
zip_ref.close()


In [ ]:
base_dir = 'cats_and_dogs_filtered/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

In [ ]:
train_cats_dir = os.path.join(train_dir,'cats')
train_dogs_dir = os.path.join(train_dir,'dogs')
validation_cats_dir = os.path.join(validation_dir,'cats')
validation_dogs_dir = os.path.join(validation_dir,'dogs')

In [ ]:
train_cat_fnames = os.listdir(train_cats_dir)
train_dog_fnames = os.listdir(train_dogs_dir)
validation_cats_fnames = os.listdir(validation_cats_dir)
validation_dogs_fnames = os.listdir(validation_dogs_dir)

In [ ]:
nrows= 4
ncols = 4
pic_index =0
fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)
pic_index=8
next_cat = [os.path.join(train_cats_dir, fname) for fname in train_cat_fnames[pic_index-8:pic_index]]
next_dog = [os.path.join(train_dogs_dir, fname) for fname in train_dog_fnames[pic_index-8:pic_index]]

for i,  img_path in enumerate(next_cat+next_dog):
  sp = plt.subplot(nrows,ncols, i+1)
  sp.axis("Off")
  img =mpimg.imread(img_path)
  plt.imshow(img)


### Build CNN Model Using Keras Layers Objects

#### Build CNN Architecture

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras

In [ ]:
img_input = keras.layers.Input(shape=(150,150,3))

#Embedding Layers
L = keras.layers.Conv2D(16,3, activation='relu')(img_input)
L = keras.layers.MaxPooling2D(2)(L)

L = keras.layers.Conv2D(32,3, activation='relu')(L)
L = keras.layers.MaxPooling2D(2)(L)

L = keras.layers.Conv2D(64,3, activation='relu')(L)
L = keras.layers.MaxPooling2D(2)(L)

#Fully Connected Layer
F = keras.layers.Flatten()(L)
##Classifier
F = keras.layers.Dense(512, activation='relu')(F)
output = keras.layers.Dense(1, activation='sigmoid')(F)

In [ ]:
my_cnn_model = keras.Model(img_input,output)

In [ ]:
my_cnn_model.summary()

#### Train CNN

In [ ]:
from keras.optimizers import RMSprop

##### Model Compile



In [ ]:
my_cnn_model.compile(loss='binary_crossentropy',
                     optimizer=RMSprop(learning_rate=.001),
                     metrics=['acc'])

##### Preprocess Image Data

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#Rescaling
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen   = ImageDataGenerator(rescale=1./255)

#Fetch Data and Resize
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size =(150,150),
                                                    batch_size  =20,
                                                    class_mode  ='binary')
validation_generator = val_datagen.flow_from_directory(validation_dir,
                                                        target_size =(150,150),
                                                        batch_size  =20,
                                                        class_mode  ='binary')


##### Model Fit

In [ ]:
history = my_cnn_model.fit(train_generator,
                           steps_per_epoch=100,
                           epochs=15,
                           validation_data=validation_generator,
                           verbose=2)

#### Feature Mapping

In [ ]:
model = my_cnn_model

In [ ]:
import numpy as np
import random
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras import Model

In [ ]:
# Let's define a new Model that will take an image as input, and will output
# intermediate representations for all layers in the previous model after
# the first.
successive_outputs = [layer.output for layer in model.layers[1:]]
visualization_model = Model(img_input, successive_outputs)

# Let's prepare a random input image of a cat or dog from the training set.
cat_img_files = [os.path.join(train_cats_dir, f) for f in train_cat_fnames]
dog_img_files = [os.path.join(train_dogs_dir, f) for f in train_dog_fnames]
img_path = random.choice(cat_img_files + dog_img_files)

img = load_img(img_path, target_size=(150, 150))  # this is a PIL image
x = img_to_array(img)  # Numpy array with shape (150, 150, 3)
x = x.reshape((1,) + x.shape)  # Numpy array with shape (1, 150, 150, 3)

# Rescale by 1/255
x /= 255

# Let's run our image through our network, thus obtaining all
# intermediate representations for this image.
successive_feature_maps = visualization_model.predict(x)

# These are the names of the layers, so can have them as part of our plot
layer_names = [layer.name for layer in model.layers]

# Now let's display our representations
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  if len(feature_map.shape) == 4:
    # Just do this for the conv / maxpool layers, not the fully-connected layers
    n_features = feature_map.shape[-1]  # number of features in feature map
    # The feature map has shape (1, size, size, n_features)
    size = feature_map.shape[1]
    # We will tile our images in this matrix
    display_grid = np.zeros((size, size * n_features))
    for i in range(n_features):
      # Postprocess the feature to make it visually palatable
      x = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std()
      x *= 64
      x += 128
      x = np.clip(x, 0, 255).astype('uint8')
      # We'll tile each filter into this big horizontal grid
      display_grid[:, i * size : (i + 1) * size] = x
    # Display the grid
    scale = 20. / n_features
    plt.figure(figsize=(scale * n_features, scale))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

### Improved Model

In [ ]:
# Improved CNN with Image Augmentation and Transfer Learning

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras

In [ ]:
#mage Preprocessing and Augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#Constants 
BATCH_SIZE = 20
IMG_SHAPE = 150

In [ ]:
image_gen_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2, 
    horizontal_flip = True,
    fill_mode = 'nearest'
)
train_data_gen = image_gen_train.flow_from_directory(
    batch_size=BATCH_SIZE,
    directory=train_dir,
    shuffle=True,
    target_size=(IMG_SHAPE, IMG_SHAPE),
    class_mode='binary'
)



In [ ]:
image_gen_val = ImageDataGenerator(rescale=1./255)
val_data_gen = image_gen_val.flow_from_directory(
    batch_size = BATCH_SIZE,
    directory = validation_dir,
    shuffle=True,
    target_size = (IMG_SHAPE,IMG_SHAPE),
    class_mode = 'binary'
)

In [ ]:
def plotImages(images_arr):
  fig, axes = plt.subplots(1,len(images_arr), figsize=(20,20))
  axes =axes.flatten()
  for img,ax in zip(images_arr,axes):
    ax.imshow(img)
  plt.tight_layout()
  plt.show()

In [ ]:
augmented_images = [train_data_gen[0][0][12] for i in range(6)]

In [ ]:
plotImages(augmented_images)

In [ ]:
## Transfer Learning

In [ ]:
from keras.applications import vgg16

In [ ]:
#vgg16 Pre-Trained Model 

In [ ]:
vgg = vgg16.VGG16(include_top = False, weights='imagenet', input_shape =(150,150,3))

In [ ]:
vgg.summary()

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, InputLayer

In [ ]:
output = vgg.layers[-1].output
output = Flatten()(output)
vgg_model = keras.Model(vgg.input, output)
vgg_model.trainable = False

In [ ]:
for layer in vgg_model.layers:
    layer.trainable = False

In [ ]:
#Build CUstomer Classified

In [ ]:
model = keras.Sequential()
model.add(vgg_model)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.15))
model.add(Dense(64, activation='relu'))


model.add(Dense(1, activation = 'sigmoid'))



In [ ]:
model.summary()

In [ ]:
#Model Compiler

In [ ]:
model.compile(
    loss = "binary_crossentropy",
    optimizer = keras.optimizers.RMSprop(learning_rate=0.0005), 
    metrics = ['binary_accuracy']

)

In [ ]:
#Early Stopping Callback

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)


In [ ]:
history = model.fit(
    train_data_gen, 
    steps_per_epoch = 100,
    epochs = 5,
    validation_data = val_data_gen,
    validation_steps = 50,
    callbacks = [callback],
    verbose = 2
)

In [ ]:
acc = history.history['binary_accuracy']
val_acc = history.history['val_binary_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(20, 12))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
from keras.preprocessing.image import load_img, img_to_array

In [ ]:
def cat_or_dog(file):
    img = load_img(file, target_size=(IMG_SHAPE,IMG_SHAPE))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)

    probs = model.predict(img)

    if probs[0][0]==1:
        result = 'dog'
    else:
        result = 'cat'

    plt.imshow(plt.imread(file))
    plt.show()

    print("This is clearly a " + result + "!")
    return probs

In [ ]:
cat_or_dog('image.jpg')

In [ ]:
cat_or_dog('doge.jpg')

In [ ]:
cat_or_dog('dog.jpg')

In [ ]:
#Time Series Analysis using ARIMA

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
DF =pd.read_csv("Electric_Production.csv")

In [ ]:
DF

In [ ]:
DF = DF.rename(columns={'DATE':'ds', 'IPG2211A2N': 'ts'})

In [ ]:
DF.head()

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(DF.ts)

In [ ]:
## Signal Decomposition (Trend, Seasonality and Residual)

In [ ]:
#Log Transform

In [ ]:
def log_transform(df, ts): return df[ts].apply(lambda x:np.log(x))

In [ ]:
DF['ts_log'] = log_transform(DF, ts = 'ts')


In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
decomposition = seasonal_decompose(DF['ts'],period = 4, extrapolate_trend=3)

In [ ]:
def plot_decomposition(df, ts, trend,seasonal, residual):
  f, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2,2, figsize=(15,8), sharex=True )

  ax1.plot(df[ts], label="Original")
  ax1.legend(loc='best')
  ax1.tick_params(axis ='x', rotation=45)

  ax2.plot(df[trend], label="Trend")
  ax2.legend(loc='best')
  ax2.tick_params(axis ='x', rotation=45)

  ax3.plot(df[seasonal], label="Seasonal")
  ax3.legend(loc='best')
  ax3.tick_params(axis ='x', rotation=45)

  ax4.plot(df[residual], label="Residuals")
  ax4.legend(loc='best')
  ax4.tick_params(axis ='x', rotation=45)

  plt.tight_layout()
  plt.show()

In [ ]:
decomposition = seasonal_decompose(DF['ts'],period = 12, extrapolate_trend=3)

In [ ]:
New_DF = DF.copy()
New_DF.loc[:,'trend']=decomposition.trend
New_DF.loc[:,'seasonal']=decomposition.seasonal
New_DF.loc[:, 'residual'] = decomposition.resid


In [ ]:
New_DF

In [ ]:
plot_decomposition(New_DF, ts='ts', trend='trend', seasonal='seasonal', residual='residual')


In [ ]:
## Stationarity Test

In [ ]:
## Stationairy Test: Dickey Fuller Test

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
dftest = adfuller(New_DF['residual'], autolag = 'AIC')

In [ ]:
dftest = adfuller(New_DF['residual'], autolag='AIC')


In [ ]:
def test_stationarity(df, ts):

  rolmean = df[ts].rolling(window=12, center= False).mean()
  rolstd =  df[ts].rolling(window=12, center = False).std()

  orig = plt.plot(df[ts], color = 'blue', label ="Original")
  mean = plt.plot(rolmean, color ='red', label ="Rolling Mean")
  std = plt.plot(rolstd, color='black', label ="Rolling Std")
  plt.legend(loc = 'best')
  plt.title("Rolling Mean and Standard Deviation for  %s" %(ts))
  plt.xticks(rotation =45)
  plt.show(block = False)
  plt.close

  print('Results:')
  dftest = adfuller(df[ts], autolag='AIC')
  dfoutput = pd.Series(dftest[0:4], index=["Test Statistic",'p=value','# Lasgs Used',' Number of Observations'])

  for key, value in dftest[4].items():
    dfoutput['Critical Value(%s)' %key]= value

  print(dfoutput)

In [ ]:
test_stationarity(df=New_DF, ts='residual')

In [ ]:
test_stationarity(df=DF, ts = 'ts')

In [ ]:
## Autocorrelation and Partial Autocorrelation  Plots

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Create figure
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12,8))

# Plot the ACF of df
plot_acf(New_DF['residual'], lags=10, zero=False, ax=ax1);

# Plot the PACF of df
plot_pacf(New_DF['residual'], lags=10, zero=False, ax=ax2);

In [ ]:
##Forecasting using ARIMA

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
def run_Arima(df,ts,p,d,q):
  model =ARIMA(df[ts], order=(p,d,q))
  results_arima=model.fit()

  len_results =len(results_arima.fittedvalues)
  ts_modified =df[ts][-len_results:]

  rss = sum((results_arima.fittedvalues-ts_modified)**2)
  rmse = np.sqrt(rss/len(df[ts]))
  print("RMSE: ",rmse)

  plt.figure()
  plt.plot(df[ts])
  plt.plot(results_arima.fittedvalues, color='red')
  plt.show()

  return results_arima

In [ ]:
New_DF = New_DF.fillna(value=0)

In [ ]:
Model_AR = run_Arima(df = New_DF, ts='residual', p=15, d=0, q=0)

In [ ]:
my_forecast = Model_AR.forecast(48)

In [ ]:
plt.figure()
plt.plot(New_DF['residual'])
plt.plot(my_forecast, color='green')

In [ ]:
Model_AR.summary()

In [ ]:
Model_AR.conf_int(alpha = 0.05)

In [ ]:
from statsmodels.graphics.tsaplots import plot_predict

In [ ]:
fig, ax = plt.subplots()
ax = New_DF['residual'].plot(ax=ax)
plot_predict(Model_AR,390,450, ax=ax)
plt.show()